In [ ]:
# !pip install keras_cv_attention_models

In [1]:
import evals
from tensorflow import keras
import tensorflow as tf
import IJB_evals
import matplotlib.pyplot as plt
import keras_cv_attention_models
import GhostFaceNets, GhostFaceNets_with_Bias

2024-05-10 10:56:24.867249: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 10:56:24.867304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 10:56:24.869193: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-10 10:56:24.877648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 10:56:25.656761: W tensorflow/compiler/tf2

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-05-10 10:56:31.690146: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 10:56:31.731980: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 10:56:31.732333: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

2 ways to load the model

Stride 2

In [3]:
#Either
basic_model = keras.models.load_model('checkpoints/ghostnetv1_w1.3_s2_basic_agedb_30_epoch_51_0.578333.h5', compile=False)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6


2024-05-10 10:56:37.105819: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 10:56:37.291243: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 10:56:37.291569: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/lfw.bin', batch_size=256, flip=True, PCA_acc=True)
ee.on_epoch_end(0)

Evaluating lfw: 100%|███████████████████████████████████████████████████████████████████████████████████| 47/47 [07:56<00:00, 10.14s/it]


ValueError: x and y arrays must be equal in length along interpolation axis.

In [5]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cfp_fp.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

Evaluating cfp_fp: 100%|███| 55/55 [02:25<00:00,  2.65s/it]


>>>> cfp_fp evaluation max accuracy: 0.767857, thresh: 0.194484, previous max accuracy: 0.000000
>>>> Improved = 0.767857


In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/lfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/vgg2_fp.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/calfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cplfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/agedb_30.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
tt = IJB_evals.IJB_test(lambda imgs: basic_model((tf.cast(imgs, "float32") - 127.5) * 0.0078125).numpy(), data_path='path_to_IJB_Dataset/ijb-testsuite/ijb', subset='IJBB', batch_size=16)

In [ ]:
score = tt.run_model_test_single()

In [ ]:
IJB_evals.plot_roc_and_calculate_tpr([score], names=[basic_model.name + "_IJBB"], label=tt.label)

In [ ]:
tt = IJB_evals.IJB_test(lambda imgs: basic_model((tf.cast(imgs, "float32") - 127.5) * 0.0078125).numpy(), data_path='C:/Users/mohda/Downloads/ijb-testsuite/ijb', subset='IJBC', batch_size=16)

In [ ]:
score = tt.run_model_test_single()

In [ ]:
IJB_evals.plot_roc_and_calculate_tpr([score], names=[basic_model.name + "_IJBC"], label=tt.label)

Plot face quality distribution using norm value of feature

In [ ]:
cc = tf.norm(ee.embs, axis=1).numpy()
_ = plt.hist(cc, bins=512, alpha=0.5, label='agedb_30 quality')
cc = tf.norm(tt.embs, axis=1).numpy()
_ = plt.hist(cc, bins=512, alpha=0.5, label='IJBC quality')
plt.legend()
plt.tight_layout()